<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-23 12:33:58
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 40.51 K (0.29%)
Current PnL: -33.16 L (-20.73%)
CY Booked + Current PnL: -17.48 L (-10.93%)
-------------------
Total profit:  1.36 L
Total loss:  -34.52 L
-------------------
Total Booked + Current PnL: 9.50 L (7.14%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.1%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.77 L (70.62%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.1%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,3.69,5.53,15.20,21.57,15716.0,5415.0,103395.0,397.04,-9.21,62.0,H-SC,6.20,167.0,0.34,0.74,50.88,XR,ATH,FINANCE
18,COALINDIA,-0.79,3.33,13.88,17.67,21580.0,5014.0,155474.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.81,XY25,ATH,MINING
77,TTKPRESTIG,-0.95,-31.20,45.48,0.09,31535.0,-31439.0,69338.0,770.00,60.94,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
36,ICICIGI,0.81,3.39,15.43,19.34,31923.0,6782.0,206891.0,2252.93,-12.97,62.0,X-MC,1.96,66.0,0.21,1.49,19.87,X40,ATH,INSURANCE
51,MEDANTA,2.67,-3.13,28.08,24.08,34533.0,-3969.0,122981.0,1486.00,-3.77,54.0,X-SC,9.80,87.0,-0.11,0.89,12.64,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,4.68,9.85,32.52,45.57,93455.0,25757.0,287376.0,7934.00,-29.43,68.0,H-MC,5.87,125.0,0.28,2.07,27.61,AR,NTT,ELECTRICAL
76,TRIDENT,4.35,-28.67,80.18,28.51,52759.0,-26454.0,65801.0,48.00,-8.24,50.0,M-SC,8.45,223.0,-0.50,0.47,12.60,XR,NTT,TEXTILES
50,MASFIN,3.69,5.53,15.20,21.57,15716.0,5415.0,103395.0,397.04,-9.21,62.0,H-SC,6.20,167.0,0.34,0.74,50.88,XR,ATH,FINANCE
51,MEDANTA,2.67,-3.13,28.08,24.08,34533.0,-3969.0,122981.0,1486.00,-3.77,54.0,X-SC,9.80,87.0,-0.11,0.89,12.64,XY24,NTT,HEALTHCARE
7,ATULAUTO,2.57,-15.92,73.13,45.57,121213.0,-31382.0,165750.0,844.00,3591.89,56.0,M-SC,3.93,248.0,-0.26,1.19,25.43,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,-3.10,-32.14,94.56,32.03,84595.0,-42373.0,89462.0,9161.00,-15.15,31.0,H-SC,8.62,158.0,-0.50,0.64,0.20,OX40N,NTT,IT
23,DIXON,-3.07,-25.59,76.40,31.26,131188.0,-59056.0,171712.0,18931.72,-66.76,39.0,X-MC,24.73,54.0,-0.45,1.24,5.38,X40N,ATH,IT
84,WIPRO,-2.28,-15.89,104.80,72.27,149372.0,-26917.0,142531.0,420.00,-24.09,23.0,M-LC,6.94,98.0,-0.18,1.03,0.00,XR,NTT,IT
64,SHALBY,-2.09,-41.47,113.73,25.10,120587.0,-75114.0,106029.0,327.00,679.90,33.0,M-SC,18.44,236.0,-0.62,0.76,0.00,XY24,NTT,HEALTHCARE
69,SURYODAY,-2.01,-31.85,80.53,23.03,98281.0,-57028.0,122043.0,216.00,33.77,27.0,H-SC,4.49,166.0,-0.58,0.88,21.09,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.33,2.64,102.57,107.91,155429.0,3891.0,151535.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.01,OX40N,ATH,CABLES
66,SIS,1.08,-31.10,76.00,21.26,58140.0,-34532.0,76500.0,528.00,1795.85,33.0,H-SC,4.59,168.0,-0.59,0.55,5.46,OX40N,NTT,MISC
77,TTKPRESTIG,-0.95,-31.20,45.48,0.09,31535.0,-31439.0,69338.0,770.00,60.94,29.0,M-SC,10.09,250.0,-1.00,0.50,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,-3.10,-32.14,94.56,32.03,84595.0,-42373.0,89462.0,9161.00,-15.15,31.0,H-SC,8.62,158.0,-0.50,0.64,0.20,OX40N,NTT,IT
40,INDIGOPNTS,1.18,-32.15,47.42,0.02,56160.0,-56127.0,118432.0,1408.00,80.80,30.0,M-SC,10.33,222.0,-1.00,0.85,3.02,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.79,3.33,13.88,17.67,21580.0,5014.0,155474.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.81,XY25,ATH,MINING
26,FINCABLES,2.33,2.64,102.57,107.91,155429.0,3891.0,151535.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.01,OX40N,ATH,CABLES
50,MASFIN,3.69,5.53,15.20,21.57,15716.0,5415.0,103395.0,397.04,-9.21,62.0,H-SC,6.20,167.0,0.34,0.74,50.88,XR,ATH,FINANCE
1,ABB,4.68,9.85,32.52,45.57,93455.0,25757.0,287376.0,7934.00,-29.43,68.0,H-MC,5.87,125.0,0.28,2.07,27.61,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,3.69,5.53,15.20,21.57,15716.0,5415.0,103395.0,397.04,-9.21,62.0,H-SC,6.20,167.0,0.34,0.74,50.88,XR,ATH,FINANCE
1,ABB,4.68,9.85,32.52,45.57,93455.0,25757.0,287376.0,7934.00,-29.43,68.0,H-MC,5.87,125.0,0.28,2.07,27.61,AR,NTT,ELECTRICAL
26,FINCABLES,2.33,2.64,102.57,107.91,155429.0,3891.0,151535.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.03,1.09,14.01,OX40N,ATH,CABLES
39,INDIAMART,1.11,-5.03,117.68,106.72,137839.0,-6206.0,117130.0,4810.62,-57.23,50.0,H-SC,7.95,140.0,-0.05,0.84,17.17,AR,ATH,MISC
85,ZYDUSLIFE,0.06,-4.96,42.70,35.63,85053.0,-10387.0,199187.0,1286.17,-17.12,50.0,H-MC,5.09,120.0,-0.12,1.43,10.89,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.99,-9.71,48.68,34.24,137522.0,-30386.0,282502.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.00,X40,NTT,IT
33,HCLTECH,-0.45,-7.20,33.44,23.83,75076.0,-17414.0,224510.0,1908.19,-2.40,24.0,X-LC,3.86,8.0,-0.23,1.62,7.71,X40,ATH,IT
8,AWL,-0.36,-35.89,145.18,57.19,303550.0,-117042.0,209085.0,485.00,-63.27,25.0,X-SC,17.86,81.0,-0.39,1.51,0.00,XY24,NTT,FMCG
73,TCS,-0.08,-23.07,60.64,23.58,201820.0,-99813.0,332816.0,4311.58,-33.14,27.0,X-LC,6.16,2.0,-0.49,2.40,0.23,X40,ATH,IT
53,PGHH,-0.35,-14.12,53.71,32.01,98491.0,-30144.0,183376.0,17616.87,-41.89,31.0,X-MC,9.57,57.0,-0.31,1.32,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1.98,3.33,16.88,20.77,40839.0,7791.0,241935.0,1644.00,-8.32,56.0,X-MC,0.62,64.0,0.19,1.74,10.98,X40N,NTT,BREWERIES
21,DABUR,0.61,0.26,43.33,43.70,107765.0,640.0,248708.0,735.00,-8.94,51.0,X-MC,1.33,70.0,0.01,1.79,15.35,XY24,BTT,FMCG
36,ICICIGI,0.81,3.39,15.43,19.34,31923.0,6782.0,206891.0,2252.93,-12.97,62.0,X-MC,1.96,66.0,0.21,1.49,19.87,X40,ATH,INSURANCE
65,SIEMENS,1.25,-12.78,43.90,25.51,71256.0,-23780.0,162315.0,4671.50,36.28,59.0,H-LC,2.03,51.0,-0.33,1.17,19.61,AR,ATH,ELECTRICAL
34,HINDUNILVR,1.10,-8.97,24.87,13.67,58196.0,-23063.0,234000.0,2922.00,-32.42,47.0,X-LC,2.84,9.0,-0.40,1.69,10.47,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,-0.31,-24.79,106.72,55.47,195271.0,-60311.0,182975.0,181.50,-35.39,33.0,X-MC,17.62,60.0,-0.31,1.32,0.00,X40N,ATH,FINANCE
8,AWL,-0.36,-35.89,145.18,57.19,303550.0,-117042.0,209085.0,485.00,-63.27,25.0,X-SC,17.86,81.0,-0.39,1.51,0.00,XY24,NTT,FMCG
53,PGHH,-0.35,-14.12,53.71,32.01,98491.0,-30144.0,183376.0,17616.87,-41.89,31.0,X-MC,9.57,57.0,-0.31,1.32,0.00,X40,ATH,FMCG
42,INFY,-1.99,-9.71,48.68,34.24,137522.0,-30386.0,282502.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.00,X40,NTT,IT
12,BATAINDIA,0.23,-47.51,157.56,35.20,106424.0,-61125.0,67545.0,2096.00,-11.48,33.0,X-SC,21.80,91.0,-0.57,0.49,0.23,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.15,-31.95,109.90,42.84,48618.0,-20768.0,44238.0,424.00,-55.37,45.0,X-SC,17.87,80.0,-0.43,0.32,6.41,XY24,NTT,MISC
58,RELAXO,-0.66,-53.95,230.71,52.31,154237.0,-78307.0,66853.0,1176.00,-50.92,38.0,X-SC,14.08,92.0,-0.51,0.48,0.23,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.23,-47.51,157.56,35.20,106424.0,-61125.0,67545.0,2096.00,-11.48,33.0,X-SC,21.80,91.0,-0.57,0.49,0.23,X40,NTT,FOOTWEAR
51,MEDANTA,2.67,-3.13,28.08,24.08,34533.0,-3969.0,122981.0,1486.00,-3.77,54.0,X-SC,9.80,87.0,-0.11,0.89,12.64,XY24,NTT,HEALTHCARE
35,HONAUT,1.63,-22.88,85.85,43.32,107828.0,-37272.0,125600.0,58357.33,-32.25,42.0,X-SC,11.55,90.0,-0.35,0.90,1.98,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.69,-14.46,39.18,19.06,114004.0,-49190.0,290976.0,452.00,-53.09,48.0,X-LC,17.92,1.0,-0.43,2.10,5.36,X40,NTT,FMCG
73,TCS,-0.08,-23.07,60.64,23.58,201820.0,-99813.0,332816.0,4311.58,-33.14,27.0,X-LC,6.16,2.0,-0.49,2.40,0.23,X40,ATH,IT
42,INFY,-1.99,-9.71,48.68,34.24,137522.0,-30386.0,282502.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.03,0.00,X40,NTT,IT
75,TMPV,0.61,-16.00,58.73,33.33,160728.0,-52128.0,273672.0,600.00,-82.53,58.0,X-LC,2.88,4.0,-0.32,1.97,11.88,XY24,NTT,AUTO
33,HCLTECH,-0.45,-7.20,33.44,23.83,75076.0,-17414.0,224510.0,1908.19,-2.40,24.0,X-LC,3.86,8.0,-0.23,1.62,7.71,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.43,-32.61,97.20,32.89,50573.0,-25176.0,52030.0,1800.00,-345.10,49.0,L-MC,11.25,258.0,-0.50,0.37,43.29,XR,NTT,BANKS
67,SONACOMS,0.49,-7.50,50.37,39.09,47133.0,-7586.0,93573.0,804.02,-26.68,64.0,M-MC,9.11,193.0,-0.16,0.67,32.37,AR,ATH,AUTO
50,MASFIN,3.69,5.53,15.20,21.57,15716.0,5415.0,103395.0,397.04,-9.21,62.0,H-SC,6.20,167.0,0.34,0.74,50.88,XR,ATH,FINANCE
18,COALINDIA,-0.79,3.33,13.88,17.67,21580.0,5014.0,155474.0,478.51,24.82,51.0,L-LC,11.34,185.0,0.23,1.12,24.81,XY25,ATH,MINING
7,ATULAUTO,2.57,-15.92,73.13,45.57,121213.0,-31382.0,165750.0,844.00,3591.89,56.0,M-SC,3.93,248.0,-0.26,1.19,25.43,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-1.72,-49.56,188.84,45.70,168423.0,-87627.0,89188.0,26.40,-7.37,71.0,M-SC,19.07,204.0,-0.52,0.64,46.95,XY24,NTT,TRAVEL
7,ATULAUTO,2.57,-15.92,73.13,45.57,121213.0,-31382.0,165750.0,844.00,3591.89,56.0,M-SC,3.93,248.0,-0.26,1.19,25.43,XY24,NTT,AUTO
1,ABB,4.68,9.85,32.52,45.57,93455.0,25757.0,287376.0,7934.00,-29.43,68.0,H-MC,5.87,125.0,0.28,2.07,27.61,AR,NTT,ELECTRICAL
67,SONACOMS,0.49,-7.50,50.37,39.09,47133.0,-7586.0,93573.0,804.02,-26.68,64.0,M-MC,9.11,193.0,-0.16,0.67,32.37,AR,ATH,AUTO
82,VOLTAS,-0.91,19.91,21.09,45.20,48490.0,38178.0,229920.0,1856.00,37.26,62.0,X-MC,6.66,79.0,0.79,1.66,28.06,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.68
1,25,45.41
2,50,77.29


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.12
MC    30.10
LC    24.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.97
X40      24.01
X40N     14.77
AR        8.99
XY25      8.76
XR        8.50
OX40N     7.11
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.66
H-SC    22.13
X-LC    21.41
X-SC    11.55
M-SC    10.69
H-MC     5.01
M-MC     2.06
H-LC     1.17
L-LC     1.12
M-LC     1.03
L-SC     0.75
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.60,-13.93,52.45
FINANCE,13.04,-15.26,60.62
IT,11.48,-37.68,106.68
MISC,6.88,-35.93,91.53
ELECTRICAL,6.27,-7.75,47.07
PAINTS,5.03,-32.22,51.86
INSURANCE,4.84,-0.98,35.54
PHARMA,4.09,-3.17,35.65
AUTO,3.83,-17.09,61.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3382625.0,21
X40,1386274.0,15
AR,1322852.0,10
XR,1309785.0,12
X40N,1068629.0,10
OX40N,803460.0,10
XY25,401137.0,6
SR,302681.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3720848.0,24
M-SC,1429380.0,14
X-MC,1365671.0,15
X-LC,1299257.0,12
X-SC,1201166.0,10
H-MC,382196.0,3
L-SC,187320.0,2
M-LC,149372.0,1
M-MC,98824.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1269670.0      6
           AR         951460.0      5
           XR         842020.0      7
M-SC       XY24       825942.0      6
X-LC       X40        670682.0      6
X-SC       XY24       615606.0      4
X-MC       X40        537927.0      7
           X40N       430530.0      4
X-SC       X40N       407895.0      4
H-SC       OX40N      355017.0      4
M-SC       OX40N      352665.0      5
H-SC       SR         302681.0      2
X-LC       XY24       271713.0      2
           X40N       230204.0      2
H-MC       XY24       203688.0      1
X-MC       XY25       201208.0      2
           XY24       196006.0      2
H-MC       AR         178508.0      2
X-SC       X40        177665.0      2
M-SC       XR         176278.0      2
M-LC       XR         149372.0      1
X-LC       XY25       126658.0      2
L-SC       OX40N       95778.0      1
           XR          91542.0      1
M-SC       AR          74495.0      1
H-LC       AR          71256.0      1
M-MC       XY25        51691.0      1
L-MC       XR          50573.0      1
M-MC       AR          47133.0      1
L-LC       XY25        21580.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
